#Imports and Data

In [0]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict

from google.colab import drive
drive.mount('/content/gdrive')

datapath = '/content/gdrive/Shared drives/NLP/Project_Files/'
modelpath = '/content/gdrive/Shared drives/NLP/Project_Files/Models/EmotionClassifier'

import pickle as p

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize

from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

In [0]:
1.1.lower()

AttributeError: ignored

In [0]:
train = pd.read_csv(datapath + 'train.csv')
# data = data.dropna()
train[:10]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,hit:0_conv:1,1,sadness,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN,NaN,NaN,NaN
1,hit:0_conv:1,2,sadness,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN,NaN,NaN,NaN
2,hit:0_conv:1,3,sadness,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN,NaN,NaN,NaN
3,hit:0_conv:1,4,sadness,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN,NaN,NaN,NaN
4,hit:0_conv:1,5,sadness,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN,NaN,NaN,NaN
5,hit:0_conv:1,6,sadness,I remember going to the fireworks with my best...,0,Oh was this something that happened because of...,5|5|5_2|2|5,NaN,NaN,NaN,NaN
6,hit:1_conv:2,1,fear,i used to scare for darkness,2,it feels like hitting to blank wall when i se...,4|3|4_3|5|5,NaN,NaN,NaN,NaN
7,hit:1_conv:2,2,fear,i used to scare for darkness,3,Oh ya? I don't really see how,4|3|4_3|5|5,NaN,NaN,NaN,NaN
8,hit:1_conv:2,3,fear,i used to scare for darkness,2,dont you feel so.. its a wonder,4|3|4_3|5|5,NaN,NaN,NaN,NaN
9,hit:1_conv:2,4,fear,i used to scare for darkness,3,I do actually hit blank walls a lot of times b...,4|3|4_3|5|5,NaN,NaN,NaN,NaN


In [0]:
test = pd.read_csv(datapath + 'test.csv')
test[:10]

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:0,1,disgust,I felt guilty when I was driving home one nigh...,0.0,Yeah about 10 years ago I had a horrifying exp...,2|2|5_5|5|5,NaN
1,hit:0_conv:0,2,disgust,I felt guilty when I was driving home one nigh...,1.0,Did you suffer any injuries?,2|2|5_5|5|5,NaN
2,hit:0_conv:0,3,disgust,I felt guilty when I was driving home one nigh...,0.0,No I wasn't hit. It turned out they were drunk...,2|2|5_5|5|5,NaN
3,hit:0_conv:0,4,disgust,I felt guilty when I was driving home one nigh...,1.0,Why did you feel guilty? People really shouldn...,2|2|5_5|5|5,NaN
4,hit:0_conv:0,5,disgust,I felt guilty when I was driving home one nigh...,0.0,I don't know I was new to driving and hadn't e...,2|2|5_5|5|5,NaN
5,hit:34_conv:69,1,contentment,My mother stopped by my house one day and said...,45.0,Well_comma_ can you tell me about your experie...,5|5|5_3|2|2,NaN
6,hit:34_conv:69,2,contentment,My mother stopped by my house one day and said...,12.0,Yeah i wanted to tell you about the time i was...,5|5|5_3|2|2,NaN
7,hit:34_conv:69,3,contentment,My mother stopped by my house one day and said...,45.0,Oh my goodness_comma_ that's very scary! I hop...,5|5|5_3|2|2,NaN
8,hit:34_conv:69,4,contentment,My mother stopped by my house one day and said...,12.0,Yeah he was punished hes in jail still,5|5|5_3|2|2,NaN
9,hit:37_conv:74,1,contempt,I just broke up with my girlfriend_comma_ we w...,47.0,I there_comma_ dont know what to do_comma_ jst...,5|4|4_5|5|5,NaN


In [0]:
test.keys(), train.keys()

(Index(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx',
        'utterance', 'selfeval', 'tags'],
       dtype='object'),
 Index(['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx',
        'utterance', 'selfeval', 'tags', 'Unnamed: 8', 'Unnamed: 9',
        'Unnamed: 10'],
       dtype='object'))

In [0]:
del train['conv_id'], train['utterance_idx'], train['speaker_idx'], train['selfeval'], train['tags']
del train['Unnamed: 8'], train['Unnamed: 9'], train['Unnamed: 10']
train[:10]

,context,prompt,utterance
0,sadness,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...
1,sadness,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...
2,sadness,I remember going to the fireworks with my best...,This was a best friend. I miss her.
3,sadness,I remember going to the fireworks with my best...,Where has she gone?
4,sadness,I remember going to the fireworks with my best...,We no longer talk.
5,sadness,I remember going to the fireworks with my best...,Oh was this something that happened because of...
6,fear,i used to scare for darkness,it feels like hitting to blank wall when i se...
7,fear,i used to scare for darkness,Oh ya? I don't really see how
8,fear,i used to scare for darkness,dont you feel so.. its a wonder
9,fear,i used to scare for darkness,I do actually hit blank walls a lot of times b...


In [0]:
del test['conv_id'], test['utterance_idx'], test['speaker_idx'], test['selfeval'], test['tags']
test[:10]

,context,prompt,utterance
0,disgust,I felt guilty when I was driving home one nigh...,Yeah about 10 years ago I had a horrifying exp...
1,disgust,I felt guilty when I was driving home one nigh...,Did you suffer any injuries?
2,disgust,I felt guilty when I was driving home one nigh...,No I wasn't hit. It turned out they were drunk...
3,disgust,I felt guilty when I was driving home one nigh...,Why did you feel guilty? People really shouldn...
4,disgust,I felt guilty when I was driving home one nigh...,I don't know I was new to driving and hadn't e...
5,contentment,My mother stopped by my house one day and said...,Well_comma_ can you tell me about your experie...
6,contentment,My mother stopped by my house one day and said...,Yeah i wanted to tell you about the time i was...
7,contentment,My mother stopped by my house one day and said...,Oh my goodness_comma_ that's very scary! I hop...
8,contentment,My mother stopped by my house one day and said...,Yeah he was punished hes in jail still
9,contempt,I just broke up with my girlfriend_comma_ we w...,I there_comma_ dont know what to do_comma_ jst...


In [0]:
orig_train = train.copy()

In [0]:
train = train.drop_duplicates(subset=('prompt'), keep='first', inplace=False)
del train['utterance']
len(train), len(orig_train)

(18140, 79204)

In [0]:
orig_test = test.copy()

In [0]:
test = test.drop_duplicates(subset=('prompt'), keep='first', inplace=False)
del test['utterance']
len(test), len(orig_test)

(2494, 10754)

In [0]:
unique = []
for i in range(len(train)):
    if train.iloc[i]['context'] not in unique:
        unique.append(train.iloc[i]['context'])
print(unique)
print(len(unique))

['sadness', 'fear', 'pride in achievement', 'optimism', 'contentment', 'anger', 'contempt', 'satisfaction', 'disgust', 'amusement', "oh no. o'm sorry to here that.", nan, 'Did you call the police?', '296', "I don't blame you. I don't know anything about Horses_comma_ but congrats on winning the bet"]
15


In [0]:
print(unique)
print(len(unique))

['sadness', 'fear', 'pride in achievement', 'optimism', 'contentment', 'anger', 'contempt', 'satisfaction', 'disgust', 'amusement', "oh no. o'm sorry to here that.", nan, 'Did you call the police?', '296', "I don't blame you. I don't know anything about Horses_comma_ but congrats on winning the bet"]
15


In [0]:
train = train.dropna()
len(train)

18139

In [0]:
train[:10]

,context,prompt
0,sadness,I remember going to the fireworks with my best...
6,fear,i used to scare for darkness
12,pride in achievement,I showed a guy how to run a good bead in weldi...
17,optimism,I have always been loyal to my wife.
21,fear,A recent job interview that I had made me feel...
26,contentment,I am very happy to have been first over 300 st...
30,anger,I once lost my job and got mad.
34,sadness,One year during christmas_comma_ i did not get...
39,contempt,I was at work_comma_ and my coworker was calle...
43,fear,I was home alone. I heard a loud noise come fr...


In [0]:
print(train[train['context'] == unique[10]])
print(train[train['context'] == unique[12]])
print(train[train['context'] == unique[13]])
print(train[train['context'] == unique[14]])

                             context       prompt
4811  oh no. o'm sorry to here that.  5|5|5_5|5|5
                       context       prompt
6392  Did you call the police?  4|5|5_5|5|5
     context                                             prompt
7571     296  Yes people and pets. Too soon. So it definitel...
                                                 context       prompt
61943  I don't blame you. I don't know anything about...  5|5|5_4|4|4


In [0]:
# del train[4811], train[6392], train[7571], train[61943]
train = train.drop([4811,6392,7571,61943], axis=0)

In [0]:
test_unique = []
for i in range(len(train)):
    if train.iloc[i]['context'] not in test_unique:
        test_unique.append(train.iloc[i]['context'])
print(test_unique)
print(len(test_unique))

['sadness', 'fear', 'pride in achievement', 'optimism', 'contentment', 'anger', 'contempt', 'satisfaction', 'disgust', 'amusement']
10


In [0]:
test = test.dropna()
len(test)

2493

In [0]:
test[:10]

,context,prompt
0,disgust,I felt guilty when I was driving home one nigh...
5,contentment,My mother stopped by my house one day and said...
9,contempt,I just broke up with my girlfriend_comma_ we w...
13,amusement,I received concert tickets for Christmas.
17,sadness,i've read an article about a little newborn ba...
22,contentment,my friend's baby hurt himself yesterday i was ...
27,optimism,i really hope my husband finds a full time job...
32,anger,some guys shot my neighbour and ran into the w...
36,contentment,I'm going to see my parents soon!
40,anger,I felt very angry when a co-worker of mine pre...


In [0]:
# import nltk
# nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stop_words_en = stopwords.words('english')
def cleanText(raw_text, stopwords=False):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'html').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case t

    if stopwords:
        words = [i for i in words if i not in stop_words_en]
    
    return( " ".join(words))

In [0]:
cleanTextvec = np.vectorize(cleanText)

In [0]:
filtered_train = train.copy()
filtered_test = test.copy()

In [0]:
train, test = filtered_train.copy(), filtered_test.copy()

In [0]:
train['prompt'] = cleanTextvec(train['prompt'], False)

In [0]:
train[:10]

,context,prompt
0,sadness,i remember going to the fireworks with my best...
6,fear,i used to scare for darkness
12,pride in achievement,i showed a guy how to run a good bead in weldi...
17,optimism,i have always been loyal to my wife
21,fear,a recent job interview that i had made me feel...
26,contentment,i am very happy to have been first over studen...
30,anger,i once lost my job and got mad
34,sadness,one year during christmas comma i did not get ...
39,contempt,i was at work comma and my coworker was called...
43,fear,i was home alone i heard a loud noise come fro...


In [0]:
test['prompt'] = cleanTextvec(test['prompt'], True)

In [0]:
test[:10]

,context,prompt
0,disgust,felt guilty driving home one night person trie...
5,contentment,mother stopped house one day said saw dogs roa...
9,contempt,broke girlfriend comma together years comma fe...
13,amusement,received concert tickets christmas
17,sadness,read article little newborn babygirl died prem...
22,contentment,friend baby hurt yesterday trying cheer
27,optimism,really hope husband finds full time job soon
32,anger,guys shot neighbour ran woods
36,contentment,going see parents soon
40,anger,felt angry co worker mine pretended car accide...


In [0]:
test_unique

['sadness',
 'fear',
 'pride in achievement',
 'optimism',
 'contentment',
 'anger',
 'contempt',
 'satisfaction',
 'disgust',
 'amusement']

In [0]:
emodict = {'amusement':0, 'satisfaction':1, 'optimism':2, 'pride in achievement':3, 'contentment':4,
           'anger':5, 'fear':6, 'disgust':7, 'sadness':8, 'contempt':9}
for i in test_unique:
    print(i, emodict[i])

sadness 8
fear 6
pride in achievement 3
optimism 2
contentment 4
anger 5
contempt 9
satisfaction 1
disgust 7
amusement 0


In [0]:
train['Rating'] = 0
test['Rating'] = 0

In [0]:
for emotion in emodict:
    train['Rating'][train['context']==emotion] = emodict[emotion]
train[:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,context,prompt,Rating
0,sadness,i remember going to the fireworks with my best...,8
6,fear,i used to scare for darkness,6
12,pride in achievement,i showed a guy how to run a good bead in weldi...,3
17,optimism,i have always been loyal to my wife,2
21,fear,a recent job interview that i had made me feel...,6
26,contentment,i am very happy to have been first over studen...,4
30,anger,i once lost my job and got mad,5
34,sadness,one year during christmas comma i did not get ...,8
39,contempt,i was at work comma and my coworker was called...,9
43,fear,i was home alone i heard a loud noise come fro...,6


In [0]:
for emotion in emodict:
    test['Rating'][test['context']==emotion] = emodict[emotion]
test[:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,context,prompt,Rating
0,disgust,felt guilty driving home one night person trie...,7
5,contentment,mother stopped house one day said saw dogs roa...,4
9,contempt,broke girlfriend comma together years comma fe...,9
13,amusement,received concert tickets christmas,0
17,sadness,read article little newborn babygirl died prem...,8
22,contentment,friend baby hurt yesterday trying cheer,4
27,optimism,really hope husband finds full time job soon,2
32,anger,guys shot neighbour ran woods,5
36,contentment,going see parents soon,4
40,anger,felt angry co worker mine pretended car accide...,5


In [0]:
# train.to_csv(datapath + 'train_cleaned_stop_emotion.csv')
# test.to_csv(datapath + 'test_cleaned_stop_emotion.csv')

In [0]:
# train_without_split = train.copy()
# train = train_without_split.copy()

In [0]:
del train['context'], test['context']

In [0]:
max_features = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
maxlen = 1000
batch_size = 128
nb_classes = 10

In [0]:
# train, test = train_test_split(data, test_size = 0.2, random_state = 42)
# train, val = train_test_split(train, test_size = 0.1, random_state = 42)
y_train = np.array(train['Rating'])
# y_val = np.array(val['Rating'])
y_test = np.array(test['Rating'])

In [0]:
print(train.shape)
# print(val.shape)
print(test.shape)

(18135, 2)
(2493, 2)


In [0]:
train.to_csv(datapath + "akhilesh_train_emotion.csv", index=False)
test.to_csv(datapath + "akhilesh_test_emotion.csv", index=False)
val.to_csv(datapath + "akhilesh_val_emotion.csv", index=False)

In [0]:
len(train["prompt"])

18135

In [0]:
tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(train['prompt'])
f = open(datapath + 'akhilesh_normal_tokenizer_prompts.dat', 'wb')
p.dump(tokenizer, f)
f.close()

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [0]:
# f = open(datapath + 'tokenizer_clean_large.dat', 'rb')
# tokenizer=p.load(f)
# f.close()

In [0]:
# vectorize the text samples into a 2D integer tensor

sequences_train = tokenizer.texts_to_sequences(train['prompt'])
# sequences_val = tokenizer.texts_to_sequences(val['prompt'])
sequences_test = tokenizer.texts_to_sequences(test['prompt'])


print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
# X_val = sequence.pad_sequences(sequences_val, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)


Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_val = np_utils.to_categorical(y_val, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)



print('X_train_without shape:', X_train.shape)
# print('X_train_without shape:', X_val.shape)
print('X_test_without shape:', X_test.shape)

Pad sequences (samples x time)
X_train_without shape: (18135, 1000)
X_test_without shape: (2493, 1000)


In [0]:
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [0]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [0]:
def max_1d(X):
    return K.max(X, axis=1)

In [0]:
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D

In [0]:
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D
from keras.models import Model
from keras.layers import Input,Flatten, Dense, Embedding, RNN, Conv1D, BatchNormalization, MaxPooling1D, Activation, Dropout, concatenate, Lambda
from keras import optimizers

#CNN

In [0]:

nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2

# nb_filter *= 2

print('Build model...')
cmodel1 = Sequential()
cmodel1.add(Embedding(max_features, 1000))
cmodel1.add(SpatialDropout1D(0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
cmodel1.add(Convolution1D(nb_filter=nb_filter,
                       filter_length=filter_length,
                       border_mode='valid',
                       activation='tanh',
                       subsample_length=1))

#cmodel1.add(BatchNormalization())
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)

cmodel1.add(Lambda(max_1d, output_shape=(nb_filter,)))
cmodel1.add(Dense(hidden_dims))
cmodel1.add(Dropout(0.2))
cmodel1.add(Activation('relu'))
cmodel1.add(Dense(nb_classes))
cmodel1.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
cmodel1.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])


cmodel1.summary()


Build model...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 1000)        20000000  
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, None, 1000)        0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 300)         900300    
_________________________________________________________________
lambda_5 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_5 (Dropout)          (None, 300)               0         
_________________________________________________________________
activation_9 (Activation)    (None, 300

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`


In [0]:
print('Train...')
cmodel1.fit(X_train, Y_train, batch_size=batch_size, epochs=1,)
        #  validation_data=(X_val, Y_val))
score, acc = cmodel1.evaluate(X_test, Y_test,
                           batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/1
2493/2493 [==============================] - 1s 253us/step
Test score: 0.23123532408124417
Test accuracy: 0.922262329319802


In [0]:
# val_preds = cmodel1.predict_classes(X_val, verbose=0)
preds3 = cmodel1.predict_classes(X_test, verbose=0)
# print('Validation micro accuracy: ', accuracy_score(val['Rating'], val_preds) * 100)
print('Test micro accuracy: ', accuracy_score(test['Rating'], preds3) * 100)

Test micro accuracy:  55.43521861211392


In [0]:
# s = 0
# for i in range(10):
#     s += accuracy_score(val[val['Rating']==i]['Rating'], val_preds[val['Rating']==i])
# print('Validation macro accuracy:',s * 10)

s = 0
for i in range(10):
    s += accuracy_score(test[test['Rating']==i]['Rating'], preds3[test['Rating']==i])
print('Test macro accuracy:',s * 10)


Test macro accuracy: 54.59768553195878


In [0]:
p = r = 0

for i in range(10):
    temp = sum(test[preds3==i]['Rating'] == i) / sum(preds==i)
    p += temp
    temp = sum(test[preds3==i]['Rating'] == i) / sum(test['Rating']==i)
    r += temp
p /= 10
r /= 10
print(2*p*r/(p+r) * 100)

55.92438382026956


In [0]:
preds == 1

array([False, False, False, ...,  True, False, False])

In [0]:
# import nltk

In [0]:
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True